**STACK INSTRUCTIONS** 
- R6 will store "going" addresses while going to push/pop
- R7 will store "return" addresses while going to push/pop
- R5 will be used in push/pop to point to top of stack, so won't be saved

To call a subroutine, using push directly: 
-     LEA R6, [SUBROUTINE LABEL]
-     JSR PUSH

Before returning from a subroutine, using pop directly: 
-     LEA R0, POP
-     JMP R0

To call a subroutine, using push indirectly (NOTE: modifies R1, save or use a different register if necessary): 
-      LEA R6, [SUBROUTINE LABEL]
-      LD R1, [LABEL OF MEMORY CONTAINING .FILL PUSH]
-      JSRR R1
        
To return from a subroutine, using pop indirectly:
-      LD R0, [LABEL OF MEMORY CONTAINING .FILL POP]
-      JMP R0

Path: 
- Store address of subroutine in R6
- JSR to push, which stores starting point in R7
- Push does JMP R6 to get to subroutine
- Subroutine JMPs to pop
- Pop RETs to starting point 

Eventually, we may need to alter some of these instructions as things get too far apart for JSR or w/e.

**ROOM/STATE DATA FORMAT**

(subject to changes?)

Every room/state stored in memory has a label pointing to the beginning of its data block, and variables stored at regular distances relative to that top. 

For all rooms:
- Label + 0: which quality (HP/stats) the room changes (0 for none) [Note: Could this field also be used to change variables, e.g. whether you have a sword?]
- Label + 1: amount of quality change
- Label + 2: type of state (0 for user-input directed, 1 for dice-directed, 2 for variable-directed, 3 for game end, 4 for simple message room)

Then if user-input room: 
- Label + 3, 4, 5: addresses of destinations for 1, 2, and 3 inputs
- Label + 6: Description string in compressed format

Or if dice room: 
- Label + 3: Stat to check
- Label + 4: Difficulty of check
- Label + 5: Destination if success
- Label + 6: Destination if failure

Or if variable room:
- Label + 3: Address of variable to check 
- Label + 4: Value of that variable that counts as a success
- Label + 5: Destination if success
- Label + 6: Destination if failure

Or if end room: 
- Label + 3: Endgame message

Or if simple message room: 
- Label + 3: Unconditional destination
- Label + 4: Message to display

**STAT & VARIABLE CODES**
1. HP
2. CHA
3. VIT
4. WIS
5. LUCK
6. SWORD
7. DOG

In [2]:
.ORIG x3000
BRnzp START ;; There is some setup before the game code, so this breaks there immediately. 

;;-----------------------------------------------------------------
;; Setup for the stack 
Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET

;;-----------------------------------------------------------------
;; Setup, name and class choice
;; Room where user chooses their class year
;; Needs four choices and does not follow main game loop
;; Based off code in Printing_text (NOT Room Structure Demo)

START:    LEA R0, CC_TEXT ;; loads & prints room text
          PUTSP
          IN ;; gets user input
          
          LEA CC_CHOICE, R6  ;; Carries out class choice in subroutine
          JSR PUSH
          
          LD INTRO_IND1, R6 ;; Carries out intro to stats
          JSR PUSH
          
          LD GETNAME_IND1, R6  ;; Uses GETNAME subroutine to put user-entered name into NAME 
          JSR PUSH
          
          LDI R0, INITIAL_IND1 ;; Load initial location into LOC
          STI R0, LOC_IND1

          LD R0, LOOP_START_IND ;; Jump to start of game loop
          JMP R0
          
;; -------------------------------------------------
;; Signposts
LOOP_START_IND: .FILL LOOP_START
INITIAL_IND1: .FILL INITIAL_LOC
LOC_IND1: .FILL LOC
VIT_IND1: .FILL VIT
CHA_IND1: .FILL CHA
WIS_IND1: .FILL WIS
GETNAME_IND1: .FILL GETNAME
INTRO_IND1: .FILL INTRO
;; -------------------------------------------------
;; Subroutine for parsing input for class choice
;; Convert the ASCII value in R0 to actual hex value by subtracting 30
;; Subtract 2 from the result
;; 1-2 = -1 so if result is negative, the user input 1
;; 2-2 = 0 so if result is zero, the user input 2
;; 3-2 = 1 so if result is positive, the user input 3 or 4
CC_CHOICE:   
          LD R1, CC_ASCII   ;; Load inverted ASCII conversion constant into R1
          AND R2, R2, #0   ;; Clear R2
          ADD R0, R0, R1   ;; Add ASCII constant to convert ASCII value into a number
          ADD R0, R0, #-2
          BRn CC_CH1 ;; user chose 1
          BRz CC_CH2 ;; user chose 2
          BRp CC_CH34 ;; user chose 3 or 4
          
CC_CH1:      LEA R0, CC_TEXT1
          ADD R2, R2, #1
          ST R2, CLASS ;; sets CLASS to 1
          ST R7, CC_SAVE_R7
          PUTSP
          LD R7, CC_SAVE_R7 
          AND R3, R3, #0   ;; Set charisma to 0
          ADD R3, R3, #0
          STI R3, CHA_IND1 
          AND R3, R3, #0   ;; Set vitality to 2
          ADD R3, R3, #2
          STI R3, VIT_IND1
          AND R3, R3, #0   ;; Set wisdom to 0
          ADD R3, R3, #0
          STI R3, WIS_IND1
          LEA R0, POP
          JMP R0
CC_CH2:      LEA R0, CC_TEXT2
          ADD R2, R2, #2
          ST R2, CLASS ;; sets CLASS to 2
          ST R7, CC_SAVE_R7
          PUTSP
          LD R7, CC_SAVE_R7
          AND R3, R3, #0   ;; Set charisma to 1
          ADD R3, R3, #1
          STI R3, CHA_IND1 
          AND R3, R3, #0   ;; Set vitality to 1
          ADD R3, R3, #1
          STI R3, VIT_IND1
          AND R3, R3, #0   ;; Set wisdom to 0
          ADD R3, R3, #0
          STI R3, WIS_IND1
          LEA R0, POP
          JMP R0
CC_CH34:     ADD R0, R0, #-2
          BRn CC_CH3 ;; user chose 3
          BRzp CC_CH4 ;; user chose 4
CC_CH3:      LEA R0, CC_TEXT3
          ADD R2, R2, #3
          ST R2, CLASS ;; sets CLASS to 3
          ST R7, CC_SAVE_R7
          PUTSP
          LD R7, CC_SAVE_R7
          AND R3, R3, #0   ;; Set charisma to 1
          ADD R3, R3, #1
          STI R3, CHA_IND1 
          AND R3, R3, #0   ;; Set vitality to 0
          ADD R3, R3, #0
          STI R3, VIT_IND1
          AND R3, R3, #0   ;; Set wisdom to 1
          ADD R3, R3, #1
          STI R3, WIS_IND1          
          LEA R0, POP
          JMP R0
CC_CH4:      LEA R0, CC_TEXT4
          ADD R2, R2, #4
          ST R2, CLASS ;; sets CLASS to 4
          ST R7, CC_SAVE_R7
          PUTSP
          LD R7, CC_SAVE_R7
          AND R3, R3, #0   ;; Set charisma to 0
          ADD R3, R3, #0
          STI R3, CHA_IND1 
          AND R3, R3, #0   ;; Set vitality to 0
          ADD R3, R3, #0
          STI R3, VIT_IND1
          AND R3, R3, #0   ;; Set wisdom to 2
          ADD R3, R3, #2
          STI R3, WIS_IND1          
          LEA R0, POP
          JMP R0

;; To save R7 so PUTSP doesn't overwrite return path
CC_SAVE_R7: .FILL #0
;;;--------------------------------------------------------------------------------------
;; Data for class choice subroutine
CC_ASCII:   .FILL x-30
CLASS:   .FILL #0
CC_TEXT1:   .STRINGC "You are a freshman! +2 Vitality\n\n"
CC_TEXT2:   .STRINGC "You are a sophomore! +1 Vitality, +1 Charisma\n\n"
CC_TEXT3:   .STRINGC "You are a junior! +1 Charisma, +1 Wisdom\n\n"
CC_TEXT4:   .STRINGC "You are a senior! +2 Wisdom\n\n"
CC_TEXT:    .STRINGC "It's the start of a new day at Bryn Mawr College! What class year are you?\nIn order to choose a year, input the number next to your choice. (Note that if you input an invalid option, it will be interpreted as one of the given options.)\n1  Be a freshman.\n2  Be a sophomore.\n3  Be a junior.\n4  Be a senior.\n" 
;;-----------------------------------------------------------
;; Intro to stats
INTRO:
        LEA R0, INTRO_TEXT
        ST R7, INTRO_SAVE_R7
        PUTSP
        LEA R0, INTRO_CHA
        PUTSP
        LD R3, INTRO_ASCII
        LDI R0, CHA_IND2
        ADD R0, R0, R3
        OUT
        LEA R0, INTRO_WIS
        PUTSP
        LDI R0, WIS_IND2
        ADD R0, R0, R3
        OUT
        LEA R0, INTRO_VIT
        PUTSP
        LDI R0, VIT_IND2
        ADD R0, R0, R3
        OUT
        LEA R0, INTRO_NLINE
        PUTSP
        
        LD R7, INTRO_SAVE_R7
        LD R0, POP_IND2
        JMP R0
        
;; To save R7 so PUTSP doesn't overwrite return path
INTRO_SAVE_R7: .FILL #0
;;-----------------------------------------------------------
INTRO_TEXT: .STRINGC "Here are your stats! When an action you can choose has one of the stats listed next to it, choosing that option will require a check based on that stat -- whether you succeed or fail will affect what happens next! Various things can raise or lower your stats throughout the game. \n\n"
INTRO_CHA: .STRINGC "Charisma: "
INTRO_WIS: .STRINGC "\nWisdom: "
INTRO_VIT: .STRINGC "\nVitality: "
INTRO_NLINE: .STRINGC "\n\n"
INTRO_ASCII: .FILL x30
CHA_IND2: .FILL CHA
WIS_IND2: .FILL WIS
VIT_IND2: .FILL VIT
;;-----------------------------------------------------------
POP_IND2: .FILL POP
;;;---------------------------------------------------------
;; Subroutine for getting name from the user
;; prints room description, sets up for running NAMING
GETNAME: LEA R0, ROOM_INFO ;; load R0 w/string stored in ROOM_INFO
        PUTSP ;; print ROOM_INFO
        LEA R1, NAME ;; load R1 with NAME starting location
        LD R2, MAX_LEN

NAMING: GETC ;; get input, value stored in R0
        STR R0, R1, #0
        ADD R1, R1, #1
        ADD R0, R0, #0
        BRz FIN ;; if character is zero terminator, done
        ADD R2, R2, #-1
        BRz FULL ;; if we're out of space, done
        BRnzp NAMING

FULL: ADD R1, R1, #-1 ;; If name exceeded length, make sure to replace last char with a zero terminator
      AND R2, R2, #0 
      STR R2, R1, #0

FIN: 
    LEA R0, NAME_ECHO ;; Echo name to user in a welcome message
    PUTSP
    LEA R0, NAME
    PUTS
    LEA R0, NAME_T1
    PUTSP
    LD R0, POP_IND2
    JMP R0

NAME: .BLKW #11
MAX_LEN: .FILL #11
ROOM_INFO: .STRINGC "Give your character a name! Type a (2-10 letter) name for your character in the box, then hit return to finish.\n"
NAME_ECHO: .STRINGC "Good morning, "
NAME_T1: .STRINGC "!\n"
;;-------------------------------------------------------------------------------------
;; Game status message text data
STAT_CAP_INITIALS: .FILL #0
CAP_TREF: .FILL CAPITAL_T
CAP_CREF: .FILL CAPITAL_C
CAP_VREF: .FILL CAPITAL_V
CAP_WREF: .FILL CAPITAL_W
CAP_FREF: .FILL CAPITAL_F

CAPITAL_T: .STRINGC "T"
CAPITAL_C: .STRINGC "C" 
CAPITAL_V: .STRINGC "V"
CAPITAL_W: .STRINGC "W" 
CAPITAL_F: .STRINGC "F"

STAT_INITIALS: .FILL #0
TREF: .FILL T
CREF: .FILL C
VREF: .FILL V
WREF: .FILL W
FREF: .FILL F

T: .STRINGC "t"
C: .STRINGC "c"
V: .STRINGC "v" 
W: .STRINGC "w"
F: .STRINGC "f"

STAT_NAMES: .FILL #0 ;; Start of stat names block
HP_NAMEREF: .FILL HP_NAME
CHA_NAMEREF: .FILL CHA_NAME
VIT_NAMEREF: .FILL VIT_NAME
WIS_NAMEREF: .FILL WIS_NAME
LUCK_NAMEREF: .FILL LUCK_NAME

HP_NAME: .STRINGC "ime remaining"
CHA_NAME: .STRINGC "harisma"
VIT_NAME: .STRINGC "itality"
WIS_NAME: .STRINGC "isdom"
LUCK_NAME: .STRINGC "ortune" 

STAT_T1: .STRINGC "Your "
STAT_T2: .STRINGC " increased by "
STAT_T3: .STRINGC " decreased by "
STAT_T4: .STRINGC "!\n\n"

DICE_T1: .STRINGC " check: You rolled "
DICE_T2: .STRINGC " with a stat bonus of " 
DICE_T3: .STRINGC " and succeeded!\n\n"
DICE_T4: .STRINGC " and failed!\n\n"

STATS_ADDR: .FILL STATS
STATNAMES_ADDR: .FILL STAT_NAMES
;;-------------------------------------------------------------
;; Variables for main loop
INITIAL_LOC: .FILL A1
OUT_OF_TIME: .FILL H1
LOC: .FILL A1  
STATS: .FILL #0 ;; Start of stats block
HP: .FILL #50   ;; Stats + 1
CHA: .FILL #0   ;; Stats + 2
VIT: .FILL #0   ;; Stats + 3
WIS: .FILL #0   ;; Stats + 4
LUCK: .FILL #0  ;; Stats + 5
SWD: .FILL #0   ;; Stats + 6
DOG: .FILL #0   ;; Stats + 7
;; -----------------------------------------------------------
;; Main game loop
LOOP_START: 

    LD R0, HP
    BRp HP_LEFT
    
    ;; If user is out of time, give them HP and proceed to "endgame" section
    LD R0, HP
    ADD R0, R0, #10
    ST R0, HP
    LD R0, OUT_OF_TIME
    ST R0, LOC

HP_LEFT: 
    LD R1, LOC     ;; Load current state's address into R1 so we can access variables relative to it

    ;; Perform stat change for room, if it has one
    
    LDR R0, R1, #0 ;; Load which_quality into R0
    BRz NO_CHANGE  ;; If which_quality is zero, skip stat change

    LD R5, STATS_ADDR ;; Load the address of the stat array into R5
    ADD R2, R0, R5 ;; Add the offset in R0 to the address in R5 to get the address of the appropriate stat
    LDR R3, R2, #0 ;; Load the current value of that stat into R3
    LDR R4, R1, #1 ;; Load R4 with the amount the stat will be changed
    ADD R3, R3, R4 
    STR R3, R2, #0 ;; Store modified stat back into memory
    
    ;; Print message about stat change
    ;; If stat code is 2-4 (CHA, VIT, WIS), print generic stat increase/decrease message
    ;; If stat code is 1 or >4 (DOG, SWORD), don't print message 
    
    LDR R0, R1, #0
    ADD R0, R0, #-5
    BRzp NO_CHANGE ;; If which_quality is >4, don't print 
    
    ADD R0, R0, #4
    BRnz NO_CHANGE ;; If which quality is <2, don't print

    LEA R0, STAT_T1  ;; print "Your " 
    PUTSP
    
    LDR R2, R1, #0
     
    LEA R3, STAT_INITIALS  ;; print "[stat_name]"
    ADD R3, R3, R2
    LDR R0, R3, #0
    PUTSP
    
    LEA R3, STAT_NAMES
    ADD R3, R3, R2
    LDR R0, R3, #0
    PUTSP

    ;; Print "increased by .. " message if stat change was positive and "decreased" otherwise

    ADD R4, R4, #0
    BRn STAT_NEG
    
    LEA R0, STAT_T2  ;; print " increased by ... " 
    PUTSP
    
    ST R4, CONVERT_VAL  
    LEA R6, ASCII_CONVERT  ;; print "[value]"
    JSR PUSH
    
    LEA R0, STAT_T4   ;; print "!\n"
    PUTSP
    
    BRnzp NO_CHANGE
    
STAT_NEG:    
    LEA R0, STAT_T3  ;; print " decreased by "
    PUTSP
    
    NOT R4, R4
    ADD R4, R4, #1 ;; First negate the stat, then print
    ST R4, CONVERT_VAL
    LEA R6, ASCII_CONVERT  ;; print "[value]"
    JSR PUSH
    
    LEA R0, STAT_T4 ;; print "!\n"
    PUTSP
    
NO_CHANGE:
    LDR R2, R1, #2 ;; Load room_type into R2
    LEA R3, ROOMTYPES ;; Load pointer to ROOMTYPES array into R3
    ADD R3, R3, #1    
    ADD R2, R2, R3 ;; Add room_type to ROOMTYPES to get a reference to the correct location
    LDR R2, R2, #0
    JMP R2
    
;; Pseudo-array of pointers used to implement if/elif/elif/../else logic
;; Value of R2 (room_type) is used to select a pointer to jump to
ROOMTYPES: .FILL #0
UNINPUT_REF: .FILL UINPUT
DICE_REF: .FILL DICE
VAR_REF: .FILL VARIABLE
END_REF: .FILL ENDING
MESSAGE_REF: .FILL MESSAGE

UINPUT:            
    ;; For user-directed rooms, print description, then use subroutine that reads, parses and applies number from 1-3
    ADD R0, R1, #6 ;; Load R0 with address of room's description
    PUTSP          ;; Print room description
    
    LEA R6, CHOICE 
    JSR PUSH
    BRnzp LOOP_DONE

DICE: 
    ;; Get a roll using D16
    LEA R6, D16
    JSR PUSH
    ;; Now there is a number between 1 and 16 in D16_roll
    
    ;; Load which_stat into R2
    LDR R2, R1, #3
    
    ;; Print name of stat
    LEA R0, STAT_CAP_INITIALS
    ADD R0, R0, R2
    LDR R0, R0, #0
    PUTSP
    
    LEA R0, STAT_NAMES
    ADD R0, R0, R2
    LDR R0, R0, #0
    PUTSP
    
    ;; Print dice text (" check: You rolled ")
    LEA R0, DICE_T1
    PUTSP
    
    LD R0, D16_roll
    
    ;; Print dice message
    ST R0, CONVERT_VAL
    LEA R6, ASCII_CONVERT  ;; print "[dice outcome]"
    JSR PUSH
    
    LEA R0, DICE_T2  ;; print "+"
    PUTSP
    
    ;; Get the value of that stat
    LEA R3, STATS
    ADD R3, R3, R2
    LDR R3, R3, #0

    BRzp NOT_NEGATIVE
    ;; Print stat value
    ;; (Negate first if negative)
    NOT R3, R3
    ADD R3, R3, #1
    
    LEA R0, DASH
    PUTSP
    
    ST R3, CONVERT_VAL
    LEA R6, ASCII_CONVERT ;; print " [negated stat bonus] "
    JSR PUSH
    
    NOT R3, R3
    ADD R3, R3, #1
    
    BRnzp STAT_BONUS_PRINTED
    
DASH: .STRINGC "-"
    
NOT_NEGATIVE: 
    ST R3, CONVERT_VAL
    LEA R6, ASCII_CONVERT ;; print " [stat bonus] "
    JSR PUSH
    
STAT_BONUS_PRINTED:
    LD R0, D16_roll
    
    ;; Add it to the dice roll to get [result] + [stat bonus]
    ADD R0, R0, R3
    
    ;; Load difficulty check value into R2
    LDR R2, R1, #4
    
    ;; Negate R2 for comparison
    NOT R2, R2
    ADD R2, R2, #1
    
    ;; Add R2 to R0 to compare difficulty check to total roll
    ADD R2, R2, R0
    
    ;; Now R2 is negative if failure or zero/positive for success
    BRn FAILURE
    
    ;; If success, print that you succeeded, then load address of success location into LOC
    
    LEA R0, DICE_T3  ;; print "and succeeded!"
    PUTSP
    
    LD R2, LOC
    ADD R2, R2, #5
    BRnzp DICE_DONE

FAILURE: 
    LEA R0, DICE_T4 ;; print "and failed!"
    PUTSP

    LD R2, LOC
    ADD R2, R2, #6

DICE_DONE: 
    LDR R3, R2, #0 ;; Load new location at address R2 into R3
    ST R3, LOC     ;; Store new location in LOC 
    BRnzp LOOP_DONE

VARIABLE: 
    LDR R0, R1, #3 ;; Load VAR_ADDRESS into R0
    LDR R0, R0, #0 ;; Load value of variable at VAR_ADDRESS into R0
    LDR R2, R1, #4 ;; Load min value into R2
    NOT R2, R2     
    ADD R2, R2, #1 ;; Negate R2
    ADD R2, R2, R0 ;; Compare
    BRn VAR_FAILURE
    
    ;; Success
    LD R2, LOC
    ADD R2, R2, #5
    BRnzp VAR_DONE
    
VAR_FAILURE: 
    LD R2, LOC
    ADD R2, R2, #6

VAR_DONE:
    LDR R3, R2, #0 ;; Load new location at address R2 into R3
    ST R3, LOC     ;; Store new location in LOC 
    BRnzp LOOP_DONE
    
ENDING: 
    ADD R0, R1, #3 ;; Load R0 with address of room's description
    PUTSP          ;; Print room description
    
    BRnzp PRINT_END_MESSAGE
    
    
MESSAGE: 
    ADD R0, R1, #4 ;; Load R0 with address of room's description
    PUTSP          ;; Print description
    LEA R0, NLINE
    PUTSP          ;; Print extra newline
    ADD R0, R1, #3 ;; R0 now has address containing destination 
    LDR R0, R0, #0  ;; Go to unconditional destination
    ST R0, LOC
    BRnzp LOOP_DONE
    
LOOP_DONE: 
    BRnzp LOOP_START   ;; Repeat main game loop

;; ------------------------------------------------------------------
;; Signposts to PUSH and POP for subroutines below

PUSH_IND1: .FILL PUSH
POP_IND1: .FILL POP

NLINE: .STRINGC "\n"

;; ------------------------------------------------------------------
PRINT_END_MESSAGE:
    LEA R0, END_T1
    PUTSP
    
    LEA R0, ASCII_GOOSE
    PUTSP
    HALT        
    
END_T1: .STRINGC "~*~*~ THE END ~*~*~\n" 

;; ------------------------------------------------------------------
;; Subroutine to get a number from 1-3 from the user and move user to corresponding location
;; Copied from Anita's code in printing_text

CHOICE:   
          IN
          LD R2, LOC
          LD R1, ASCII     ;; Load inverted ASCII conversion constant into R1
          ADD R0, R0, R1   ;; Add it to convert ASCII value into a number
          ADD R0, R0, #-2
          BRn CH1 ;; user chose 1
          BRz CH2 ;; user chose 2
          BRp CH3 ;; user chose 3
          
CH1:      ADD R2, R2, #3
          BRnzp CH_DONE
CH2:      ADD R2, R2, #4
          BRnzp CH_DONE
CH3:      ADD R2, R2, #5
          BRnzp CH_DONE
          
CH_DONE:  LDR R3, R2, #0 ;; Load the data at R2, aka the address of the #[choice] room, into R3
          ST LOC, R3     ;; Store that data in LOC, moving player there 
          LD R0, POP_IND1
          JMP R0

;; Data
ASCII:   .FILL x-30

;;; -----------------------------------------------------
;;; After this runs, there is a new number between 1 and 16 in D16_roll
D16: 
    ;; Call Random, loading a new random number into X
    LEA R6, Random
    LD R1, PUSH_IND1
    JSRR R1
    
    LD R0, X
    AND R0, R0, #15  ;; Mod number by 16
    ADD R0, R0, #1   ;; Add one to get number between 1 and 16
    ST R0, D16_roll
    
    LD R0, POP_IND1
    JMP R0
D16_roll: .FILL #0

;; -------------------------------------------------------
;; Subroutine to convert binary value into ASCII decimal and print.
;; Note: Algorithm does not work reliably on 3-digit or larger numbers, or negative numbers.

ASCII_CONVERT:
    LD R0, CONVERT_VAL
    ST R0, TEMPX
    ST R0, TEMPY
    
ASCII_BASE: ;; Base case: if val < #10, add x30 and print

    ;; reset TEMPZ to #-1
    LD R1, NEG_ONE
    ST R1, TEMPZ
    
    ;; check val < #10
    ADD R1, R0, #-10
    BRzp ASCII_LOOP ;; if val >= #10, skip to main loop
    
    ;; convert R1 to ASCII and print
    LD R1, ASCII_POS
    ADD R0, R0, R1
    OUT R0
    
    LD R0, POP_IND1
    JMP R0
    
ASCII_LOOP_REP: ;; start here if repeating non-base case

    ST R2, TEMPY ;; stores TEMPY from last loop

ASCII_LOOP: ;; Non-base case
    
    ;; TEMPZ = z*10
    LD R0, TEMPZ ;; R0 <- z
    LD R1, TEN ;; R1 <- 10
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1   = z*10
    ST R2, TEMPZ
    
    ;; R2 = y/10
    LD R0, TEMPY ;; R0 <- y
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    LD R2, QUOTIENT  ; R2 <- R0 / R1       = y/10
    ;;ST R2, TEMPY
    ;;ADD R2, R2, #0
    
    BRp ASCII_LOOP_REP ;; if quotient is positive, repeat loop
    
    ;; else if quotient is zero, use previous quotient (current val of TEMPY) as y
    
    ;; z = z/10 since the loop has to go one extra time to hit zero
    LD R0, TEMPZ ;; R0 <- z
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    LD R2, QUOTIENT  ; R2 <- R0 / R1    = z/10
    LD R0, QUOTIENT
    
    ;; R2 = y*z
    LD R1, TEMPY ;; R1 <- y
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1     = y*z
    
    LD R3, ASCII_POS
    ADD R0, R1, R3 ;; convert y to ASCII
    OUT R1
    
    LD R0, TEMPX
    ADD R0, R0, R2 ;; x-z: current state of val - (10^n * y)
    ST R0, TEMPX
    ST R0, TEMPY
    
    BRnzp ASCII_BASE ;; return to ASCII_BASE to check for base case
    
ASCII_POS:   .FILL x30
CONVERT_VAL:   .FILL #0 ;; value to be converted; set before calling
NEG_ONE:   .FILL #-1
TEN:   .FILL #10
TEMPX:   .FILL #0 ;; stores present state of value to be converted, after subtraction
TEMPY:   .FILL #0 ;; stores present state of value to be converted, after division
TEMPZ:   .FILL #-1 ;; stores (negative) exponent of 10

;;; -----------------------------------------------------
;;; Subroutine to generate a pseudorandom number; memory X has next random number after this runs
Random: 
    LD R0, M
    LD R1, A
    
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    
    ;; q = m / a
    LD R0, QUOTIENT     ; R0 / R1
    ST R0, Q 
    ;; r = m mod a
    LD R0, REMAINDER    ; R0 mod R1
    ST R0, R
        ;; x / q
    LD R0, X
    LD R1, Q
    
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    
    LD R1, QUOTIENT
    ST R1, TEMP2
    LD R1, REMAINDER    ; x mod q
    ST R1, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, A
    
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1
    
    ST R2, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, R
    LD R1, TEMP2
    
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1          
    LD R2, MULT_PRODUCT ; R2 <- r * TEMP2
    
    NOT R2,R2           ; -R2
    ADD R2,R2,#1
    ST R2, TEMP2 
    LD R1, TEMP1
    ADD R2, R2, R1      ; TEMP1 - TEMP2
    
TEST:  BRzp DONE        ; if x < 0 then
    LD R1, M
    ADD R2, R2, R1      ; 
DONE: ST R2, X
    LD R0, POP_IND1         ;; Return
    JMP R0

A: .FILL #7             ;; a , the multiplicative constant is given
M: .FILL #32767         ;; m = 2 Ë† 15 âˆ’ 1, the modulus is given
X: .FILL #10            ;; x, the seed is given
R: .FILL #0
Q: .FILL #0
TEMP1: .FILL #0
TEMP2: .FILL #0
BACK: .FILL #0

;;; -----------------------------------------------------
;;; Mult_Product <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:
  ST R2, MULT_PRODUCT           ; Product has the product
  LD R0, POP_IND1
  JMP R0
  
MULT_PRODUCT: .FILL #0

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  LD R0, POP_IND1
  JMP R0
QUOTIENT: .FILL #0
REMAINDER: .FILL #0

;; ------------------------------------------------------------------
;; ASCII Goose
ASCII_GOOSE: .STRINGC "\n                                   ___\n                               ,-\"\"   `.\n                             ,'  _   e )`-._\n                            /  ,' `-._<.===-'\n                           /  /\n                          /  :\n              _          /   :\n (`._    _.-\"\" \"\"--..__,'    |\n <_  `-\"\"                     \\\n  <`-                          :\n   (__   <__.                  :\n     `-.   '-.__.      _.'    /\n        \      `-.__,-'    _,'\n         `._    ,    /__,-'\n            \"\"._\__,'< <____\n                 | |  `----.`.\n                 | |        \ `.\n                 : |___      \-``\n                 \   --<\n                  `.`.<\n             hjw    `-'\n"
;; ------------------------------------------------------------------

;; A Rooms (Brecon)
A1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL A2 ;; Choice 1
    .FILL A3 ;; Choice 2
    .FILL H1 ;; Choice 3
    .STRINGC "You are in your room in Brecon! Your paper is written, your teeth are brushed, and your bag is packed. All you need to do is eat lunch, print your paper, and get to class on time to turn it in. As you're leaving your room you see a fencer, notorious for being chatty. Would you like to:\n 1) Avoid them (Wisdom) \n 2) Talk to them (Charisma)\n 3) Go back to sleep\n"
    
A2: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on user input
    .FILL #4 ;; Stat for dice roll: Wisdom
    .FILL #8 ;; Difficulty of check
    .FILL A5 ;; Destination if success
    .FILL A3 ;; Destination if failure
    
A3: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #2 ;; Stat for dice roll: Charisma
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL A6 ;; Destination if success
    .FILL A4 ;; Destination if failure
    
A4:
    .FILL #1 ;; Quality changed: HP
    .FILL #-3 ;; Amount of change: -3
    .FILL #4 ;; 4: Message state
    .FILL B1 ;; Unconditional destination
    .STRINGC "Unfortunately, you got caught up in conversation and lost track of time. Now you're running late! You hurry towards the Valley of Death.\n"
    
A5:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL B1 ;; Unconditional destination
    .STRINGC "You successfully avoided what could have been a long conversation! You head towards the Valley of Death, ahead of schedule.\n"
    
A6:
    .FILL #6 ;; Quality changed: Sword
    .FILL #1 ;; Amount of change: +1
    .FILL #4 ;; 4: Message state
    .FILL A7 ;; Unconditional destination
    .STRINGC "You got a little caught up in conversation, but the fencer gave you this cool sword!\n"
    
A7: 
    .FILL #1 ;; Quality changed: HP
    .FILL #-3 ;; Amount of change: -3
    .FILL #4 ;; 4: Message state
    .FILL B1 ;; Unconditional destination
    .STRINGC "Getting that sword must be worth being behind schedule, right? You hurry towards the Valley of Death.\n"
    
;; B Rooms (Valley of Death)
;; Note: numbers slightly different than in spreadsheet

B1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL B2 ;; Choice 1
    .FILL B4 ;; Choice 2
    .FILL B5 ;; Choice 3
    .STRINGC "You've reached the Valley of Death! Its depths loom before you. Which way would you like to go?\n 1) You don't have time for lunch -- just go behind Rhoads and go straight to Canaday to print your paper.\n 2) Brave the stairs -- it's the fastest way to lunch. (Vitality)\n 3) Go around through Park -- it may be slower, but at least you don't have to climb as many stairs.\n"
    
B2:
    .FILL #1 ;; Quality changed: HP
    .FILL #5 ;; Amount of change: +5
    .FILL #4 ;; 4: Message state
    .FILL B3 ;; Unconditional destination
    .STRINGC "Going behind Rhoads means you have plenty of time to get that paper ready, but...\n"
    
B3:
    .FILL #3 ;; Quality changed: Vitality
    .FILL #-3 ;; Amount of change: -3
    .FILL #4 ;; 4: Message state
    .FILL C1 ;; Unconditional destination
    .STRINGC "...you're starting to regret skipping lunch... Ah well, too late now. You need to print your paper! \n"
    
B4:
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #3 ;; Stat for dice roll: Vitality
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #5 ;; Difficulty of check
    .FILL B6 ;; Destination if success
    .FILL B7 ;; Destination if failure
    
B5: 
    .FILL #1 ;; Quality changed: HP
    .FILL #-5 ;; Amount of change: -5
    .FILL #4 ;; 4: Message state
    .FILL D1 ;; Unconditional destination
    .STRINGC "Forget stairs -- you go through Park and take the elevator. Much better! But the detour costs you some time. You'd better hurry if you want to make it to lunch!\n"
    
B6: .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL D1 ;; Unconditional destination
    .STRINGC "Whew! That was a slog, but you made it to the top of the stairs. On to lunch!\n"
    
B7: .FILL #1 ;; Quality changed: HP
    .FILL #-3 ;; Amount of change: -3
    .FILL #4 ;; 4: Message state
    .FILL D1 ;; Unconditional destination
    .STRINGC "The stairs got you this time! You made it to the top, but it took a little longer than you might have liked. You'd better hurry if you want to make it lunch!\n"

;; C Rooms (Canaday)

C1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL C2 ;; Choice 1
    .FILL C2 ;; Choice 2
    .FILL C4 ;; Choice 3
    .STRINGC "You enter Canaday and head for the printers. But disaster strikes -- there's only one open printer, and it's out of toner! What do you do?\n 1) Talk to the Help Desk \n 2) Talk to the circulation desk \n 3) Talk to the person using the other printer. Maybe they'd be willing to pause the textbook they're printing so you can squeeze out a few pages... (Charisma)\n"
    
C2: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL C7 ;; Choice 1
    .FILL C8 ;; Choice 2
    .FILL C7 ;; Choice 3
    .STRINGC "The student worker needs you to find a book before they'll fill the toner with printer. You can find the book using which cataloguing system? \n 1) Dewey Decimal \n 2) Library of Congress \n 3) Universal Decimal Classification \n"

C3: 
    .FILL #1 ;; Quality changed: HP
    .FILL #-3 ;; Amount of change: -3
    .FILL #4 ;; 4: Message state
    .FILL C5 ;; Unconditional destination
    .STRINGC "You did not manage to find the book, and are forced to wait until the person at the next printer has finished printing out their whole textbook before you can print your paper. Now you're really running late -- you'd better book it!\n"
    
C4: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #2 ;; Stat for dice roll: Charisma
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL C5 ;; Destination if success
    .FILL C6 ;; Destination if failure

C5: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL E1 ;; Unconditional destination
    .STRINGC "You've printed out your paper and left Canaday! Now all you have to do is make it to class on time -- no sweat!\n"
    
C6:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL C5 ;; Unconditional destination
    .STRINGC "You couldn't even convince them to let you print out a few measly pages, and have to wait for them to finish printing. Now you're really running late -- you'd better book it!\n"
  
C7:
    .FILL #1 ;; Quality changed: None
    .FILL #-3 ;; Amount of change: -3
    .FILL #4 ;; 4: Message state
    .FILL C5 ;; Unconditional destination
    .STRINGC "Unfortunately, you aren't able to find the book and the student worker refuses to get you toner. You lose some time waiting for the other printer. \n"
    
C8: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: 0
    .FILL #4 ;; 4: Message state
    .FILL E1 ;; Unconditional destination
    .STRINGC "Congrats! You're able to find the book and the student worker happily gets you toner. \n"
  
;; D Rooms (Erd)

D1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL D2 ;; Choice 1
    .FILL D3 ;; Choice 2
    .FILL D4 ;; Choice 3
    .STRINGC "You reach Erdman just as the lunch rush is starting. What are you going to do first?\n 1) Try to find a seat (Wisdom) \n 2) Look for a friend (Charisma) \n 3) Get food\n"
    
D2: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #4 ;; Stat for dice roll: Wisdom
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #12 ;; Difficulty of check
    .FILL D11 ;; Destination if success
    .FILL D6 ;; Destination if failure
    
D3: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #2 ;; Stat for dice roll: Charisma
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #12 ;; Difficulty of check
    .FILL D12 ;; Destination if success
    .FILL D6 ;; Destination if failure
    
D4: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL D7 ;; Choice 1
    .FILL D8 ;; Choice 2
    .FILL D13 ;; Choice 3
    .STRINGC "You go up to hot line but they've just run out of mac & cheese. It's going to be a little while before they put out more, but you really had your heart set on that cheesy goodness. What will you do?\n 1) Wait for more mac & cheese \n 2) Just grab something and go \n 3) Forget it, who needs lunch anyway? Just go print your paper. \n"
    
D5:
    .FILL #3 ;; Quality changed: Vitality
    .FILL #+1 ;; Amount of change: +1
    .FILL #4 ;; 4: Message state
    .FILL D9 ;; Unconditional destination
    .STRINGC "You leave Erdman, full and refreshed!\n"
    
D6: 
    .FILL #1 ;; Quality changed: HP
    .FILL #-5 ;; Amount of change: -5
    .FILL #4 ;; 4: Message state
    .FILL D5 ;; Unconditional destination
    .STRINGC "It was too crowded to find anything -- after wasting some time looking, you just grab some food and get out.\n"
    
D7: 
    .FILL #4 ;; Quality changed: Wisdom
    .FILL #+1 ;; Amount of change: +1
    .FILL #4 ;; 4: Message state
    .FILL D15 ;; Unconditional destination
    .STRINGC "Eventually you get your mac & cheese, and it is delicious! It cost you some time, but it was worth it! +Wisdom\n"
    
D8:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: 0
    .FILL #4 ;; 4: Message state
    .FILL D5 ;; Unconditional destination
    .STRINGC "You give up on your dreams of mac & cheese and just grab something fast -- you're in a hurry, after all.\n"
    
D9:
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #4 ;; Stat for dice roll: Wisdom
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL D14 ;; Destination if success
    .FILL D10 ;; Destination if failure
    
D10: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: 0
    .FILL #4 ;; 4: Message state
    .FILL H1 ;; Unconditional destination
    .STRINGC "Unfortunately, on your way through the intersection you got a little confused in the crowd and got on the Blue Bus just as it was about to leave. You're on your way to Haverford -- there's no way you'll make it to class in time now! Maybe if you explain it to your professor you'll get an extension...\n"
    
D11:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: 0
    .FILL #4 ;; 4: Message state
    .FILL D5 ;; Unconditional destination
    .STRINGC "You were able to find an empty seat! You get your food and scarf it down, and then you're on the move again.\n"
    
D12: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: 0
    .FILL #4 ;; 4: Message state
    .FILL D5 ;; Unconditional destination
    .STRINGC "Thank goodness for friends! You were able to find a seat -- you get your food and scarf it down, and then you're on the move again.\n"
    
D13: 
    .FILL #3 ;; Quality changed: Vitality
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL D9 ;; Unconditional destination
    .STRINGC "You're already starting to regret not eating as you're on your way out, but it's too late now -- on to Canaday!\n"
    
D14: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL C1 ;; Unconditional destination
    .STRINGC "You got caught up in the bus crowd in the intersection and almost ended up on the Blue Bus, but managed to get past it -- that was a close one! You definitely wouldn't make it to class on time if you ended up at Haverford.\n"
    
D15:
    .FILL #1 ;; Quality changed: HP
    .FILL #-5 ;; Amount of change: -5
    .FILL #4 ;; 4: Message state
    .FILL D5 ;; Unconditional destination
    .STRINGC "While delicious, all the first years have just gotten out of ESem, so getting food takes some time. \n"
    
;; E Rooms (Dogs)

E1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL PG1 ;; Choice 1
    .FILL E2 ;; Choice 2
    .FILL H1 ;; Choice 3
    .STRINGC "The geese-chaser van is just outside Canaday! Students are gathered around to pet the border collies, and there's just enough space for you to join them. What do you do?\n 1) Ignore them -- you need to get to class! \n 2) You should really get to class, but they're soooo cute. Maybe just a few minutes... \n 3) Give in and pet the dogs. Maybe your professor will give you an extension?\n"
    
E2: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-3 ;; Amount of change: -3
    .FILL #1 ;; 1: Based on dice roll
    .FILL #2 ;; Stat for dice roll: Charisma
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL E4 ;; Destination if success
    .FILL E5 ;; Destination if failure
    
E3: ;;GHOST ROOM
    
E4: 
    .FILL #7 ;; Quality changed: Dog
    .FILL #1 ;; Amount of change: +1
    .FILL #4 ;; 4: Message state
    .FILL PG1;; Unconditional destination
    .STRINGC "The dogs are your friends now! They are very good friends and are going to accompany you to class!\n"
    
E5:
    .FILL #1 ;; Quality changed: HP
    .FILL #-10 ;; Amount of change: -10
    .FILL #4 ;; 4: Message state
    .FILL PG1;; Unconditional destination
    .STRINGC "Whoops, that was definitely more than a few minutes. You're not even sure you can get to class on time at this point...\n"
    
;; PG Rooms (pre-geese variable checks)

PG1:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #2 ;; 2: Based on variable
    .FILL DOG ;; VAR_ADDRESS ;; What variable to check
    .FILL #1 ;; VALUE       ;; What value counts as a success
    .FILL F1_DOG ;; Destination if success
    .FILL PG2 ;; Destination if failure
    
PG2:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #2 ;; 2: Based on variable
    .FILL SWD ;; VAR_ADDRESS ;; What variable to check
    .FILL #1 ;; VALUE       ;; What value counts as a success
    .FILL F1_SWD ;; Destination if success
    .FILL F1 ;; Destination if failure
    
;; F Rooms (Geese)

F1_DOG: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL F2 ;; Choice 1
    .FILL F3 ;; Choice 2
    .FILL F4_DOG ;; Choice 3
    .STRINGC "Oh no! The geese are directly across your path through Merion Green to get to Park for class. What will you do? \n 1) Go around \n 2) Try to sneak through them (Wisdom) \n 3) Get your new dog friends to help \n"

F1_SWD: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL F2 ;; Choice 1
    .FILL F3 ;; Choice 2
    .FILL F4_SWD ;; Choice 3
    .STRINGC "Oh no! The geese are directly across your path through Merion Green to get to Park for class. What will you do? \n 1) Go around \n 2) Try to sneak through them (Wisdom) \n 3) Brandish the sword that fencer gave you \n"

F1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL F2 ;; Choice 1
    .FILL F3 ;; Choice 2
    .FILL F4 ;; Choice 3
    .STRINGC "Oh no! The geese are directly across your path through Merion Green to get to Park for class. What will you do? \n 1) Go around \n 2) Try to sneak through them (Wisdom) \n 3) Try to scare them away (Charisma) \n"
    
F2: 
    .FILL #1 ;; Quality changed: HP
    .FILL #-5 ;; Amount of change: -5
    .FILL #4 ;; 4: Message state
    .FILL F9;; Unconditional destination
    .STRINGC "You were able to avoid the geese by going around, but it cost you some time.\n"
    
F3: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #4 ;; Stat for dice roll: Wisdom
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #10 ;; Difficulty of check
    .FILL F7 ;; Destination if success
    .FILL F5 ;; Destination if failure
    
F4:
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #2 ;; Stat for dice roll: Charisma
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #10 ;; Difficulty of check
    .FILL F8 ;; Destination if success
    .FILL F6 ;; Destination if failure
    
F4_DOG: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL F9 ;; Unconditional destination
    .STRINGC "In an amazing show of skill and strength, the geese-chaser dogs chase away the geese. It brings a tear to your eye. Several nearby students clap. What good, good dogs. Now on to Park!\n"
    
F4_SWD: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL F9 ;; Unconditional destination
    .STRINGC "You feel a bit silly brandishing the fencing weapon, and for a second you're worried the geese will take it as a threat display and attack you! But after a tense moment, they honk grudgingly and flap out of your way. On to Park!\n"
    
F5:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #3 ;; 3: End state
    .STRINGC "You tried to sneak through, but the geese noticed you and have adopted you as one of their own. The bright side is, you'll never have to worry about being on time to class ever again!\n The End.\n"
    
F6: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL H1 ;; Unconditional destination
    .STRINGC "You did not scare the geese away. In fact, you failed so abysmally at scaring the geese away that they scared YOU away. You took refuge in Taylor, but you don't think they're going to let you leave until they go back to the pond tonight. How are you ever going to explain this to your professor?\n"
    
F7:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL F9 ;; Unconditional destination
    .STRINGC "Whew, you made it through without being noticed. On to Park!\n"
    
F8: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL F9 ;; Unconditional destination
    .STRINGC "You ran at them waving your arms, and they all flew away. Ha, guess those geese aren't so tough after all! Your path is clear -- on to Park!\n"
    
F9:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL G1 ;; Unconditional destination
    .STRINGC "At long last, you've reached Park! Unfortunately, since they started construction you're still not totally sure how to get to your classroom... You know it's upstairs, and near the library, but you always seem to get lost on the way there.\n"
    
;; G Rooms (Park)

G1:
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: HP
    .FILL #0 ;; 0: Based on user input
    .FILL G9 ;; Choice 1
    .FILL G2 ;; Choice 2
    .FILL H1 ;; Choice 3
    .STRINGC "You're in the atrium -- which way do you go? \n 1) Forward, into the chemistry hallway \n 2) To the stairs \n 3) Back to Brecon to take a nap \n"
    
G2: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G3 ;; Choice 1
    .FILL G14 ;; Choice 2
    .FILL G1 ;; Choice 3
    .STRINGC "You're at the ground floor of the stairs. Which way do you go? \n 1) Up \n 2) Down \n 3) Back to the atrium \n"

G3: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G2 ;; Choice 1
    .FILL G4 ;; Choice 2
    .FILL G4 ;; Choice 3
    .STRINGC "There's a door at the top of the stairs, but it's locked! You can even see the library through the window -- if you could just reach it, you would know exactly how to get to your classroom. What do you want to do? \n 1) Go back down the stairs \n 2) Try the door again, just in case it came unlocked in the 0.04 seconds since you tried it last \n 3) Press your face against the glass and look mournful. Maybe someone will take pity on you? \n"

G4: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G5 ;; Choice 1
    .FILL G2 ;; Choice 2
    .FILL H1 ;; Choice 3
    .STRINGC "Unsurprisingly, it doesn't work -- but it occurs to you that the door is pretty flimsy. Maybe you could try to break it down...? You're sure Facilities would forgive you eventually. \n 1) Do it! (Vitality) \n 2) Nah. Go back down the stairs \n 3) Take a nap \n"

G5: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #1 ;; 1: Based on dice roll
    .FILL #3 ;; Stat for dice roll: Wisdom
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL G6 ;; Destination if success
    .FILL G7 ;; Destination if failure
    
G6: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL G20 ;; Unconditional destination
    .STRINGC "It worked! And the door is only a little bit broken! You hurry towards your classroom. You know the way from here -- there's no way you can be late to class now! \n"
    
G7: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL G2 ;; Unconditional destination
    .STRINGC "It didn't work, and now your shoulder really hurts... You should probably just try to find another way around, so you go back downstairs. \n"
    
G8: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL WIN ;; Unconditional destination
    .STRINGC "You made it to class on time! Hooray, you win! \n"
    
G9: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G10 ;; Choice 1
    .FILL G12 ;; Choice 2
    .FILL G1 ;; Choice 3
    .STRINGC "You are in the chemistry hallway. There are two identical doors. Where would you like to go? \n 1) Through the first door \n 2) Through the second door \n 3) Back to the atrium \n"
    
G10:
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G9 ;; Choice 1
    .FILL G11 ;; Choice 2
    .FILL GOOSE_END ;; Choice 3
    .STRINGC "Oh no! You've accidentally walked into an ongoing class. What are you going to do? \n 1) Back out quickly \n 2) Pretend you came in on purpose and assimilate with the class. You are one of them now. Better email the Registrar. \n 3) Leave and return to the geese, your real family. \n"
    
G11: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL H1 ;; Unconditional destination
    .STRINGC "You are part of this class now. You learn a lot about axolotls, but you don't get your paper turned in on time. \n"
    
G12: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G15 ;; Choice 1
    .FILL G14 ;; Choice 2
    .FILL G9 ;; Choice 3
    .STRINGC "You are in a spacious hallway. There is a door and a staircase. Where will you go? \n 1) Through the door \n 2) Down the stairs \n 3) Back the way you came \n"
    
G13: 
    .FILL #1 ;; Quality changed: HP
    .FILL #-5 ;; Amount of change: -5
    .FILL #4 ;; 4: Message state
    .FILL G16 ;; Unconditional destination
    .STRINGC "\n"
    
G14:
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-10 ;; Amount of change: -10
    .FILL #0 ;; 0: Based on user input
    .FILL G1 ;; Choice 1
    .FILL H1 ;; Choice 2
    .FILL GOOSE_END ;; Choice 3
    .STRINGC "You get lost and wander out into the daylight sometime later, dazed and confused. You are in front of Park, and you might still be able to make it to class in time. What will you do? \n 1) Try again -- back to the atrium \n 2) Go back to Brecon and take a nap \n 3) Go back and live with the geese -- at least then you don't have to worry about getting to class on time \n"
    
G15: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G17 ;; Choice 1
    .FILL G13 ;; Choice 2
    .FILL G12 ;; Choice 3
    .STRINGC "There are some comfortable-looking chairs blocking a door labeled as the geology department. What will you do? \n 1) Continue onward \n 2) Take a break in the chairs \n 3) Go back the way you came \n"

G16: 
    .FILL #3 ;; Which quality (stat/HP) it changes: Vitality
    .FILL #1 ;; Amount of change: +1
    .FILL #0 ;; 0: Based on user input
    .FILL G17 ;; Choice 1
    .FILL G12 ;; Choice 2
    .FILL G14 ;; Choice 3
    .STRINGC "You've lost a little time, but you feel rested and refreshed! What now? \n 1) Continue onward \n 2) Go back the way you came \n 3) Move the chairs and go through the door \n"
    
G17:
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G18 ;; Choice 1
    .FILL G19 ;; Choice 2
    .FILL G15 ;; Choice 3
    .STRINGC "Unfortunately, this hallway is blocked by construction, but there is a flight of stairs. What will you do? \n 1) Try to sneak through the construction (Wisdom) \n 2) Go up the stairs \n 3) Go back the way you came \n"
    
G18: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #1 ;; 1: Based on dice roll
    .FILL #4 ;; Stat for dice roll: Wisdom
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL G8 ;; Destination if success
    .FILL G27 ;; Destination if failure
    
G19:
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G20 ;; Choice 1
    .FILL G17 ;; Choice 2
    .FILL H1 ;; Choice 3
    .STRINGC "You are in the upstairs hallway! You think you know the way to class from here -- which way would you like to go? \n 1) Onwards \n 2) Back the way you came \n 3) Back to Brecon to take a nap \n"
    
G20: 
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G21 ;; Choice 1
    .FILL G22 ;; Choice 2
    .FILL G19 ;; Choice 3
    .STRINGC "Your classroom is in sight! Unfortunately, between here and there you also see Deepak, who has a habit of ambushing you to quiz you about sorting algorithms at the worst times. What will you do? \n 1) Answer his question (Wisdom) \n 2) Try to evade him (Charisma) \n 3) Go back the way you came \n"
    
G21:
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #1 ;; 1: Based on dice roll
    .FILL #4 ;; Stat for dice roll: Wisdom
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL G24 ;; Destination if success
    .FILL G25 ;; Destination if failure
    
G22:
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #1 ;; 1: Based on dice roll
    .FILL #2 ;; Stat for dice roll: Charisma
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL G23 ;; Destination if success
    .FILL G26 ;; Destination if failure
    
G23:
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL G8 ;; Unconditional destination
    .STRINGC "You successfully avoided Deepak and his sorting algorithms! Your path to class is now clear. \n"
    
G24:
    .FILL #1 ;; Quality changed: HP
    .FILL #-5 ;; Amount of change: -5
    .FILL #4 ;; 4: Message state
    .FILL G8 ;; Unconditional destination
    .STRINGC "You answered Deepak's question correctly, and he gave you $20! Your path to class is now clear, AND you have an extra $20! \n"
    
G25:
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #0 ;; 0: Based on user input
    .FILL G8 ;; Choice 1
    .FILL H1 ;; Choice 2
    .FILL GOOSE_END ;; Choice 3
    .STRINGC "You were not able to answer Deepak's question. Your path to class is now clear, but you feel shame. What will you do? \n 1) Go on to class \n 2) Go back to Brecon and take a nap to forget your shame \n 3) Drop out of school and go live with the geese \n"
    
G26:
    .FILL #1 ;; Quality changed: HP
    .FILL #-10 ;; Amount of change: -10
    .FILL #4 ;; 4: Message state
    .FILL G21 ;; Unconditional destination
    .STRINGC "You failed to avoid Deepak, and now must answer his question. \n"
    
G27:
    .FILL #1 ;; Which quality (stat/HP) it changes: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL G1 ;; Unconditional destination
    .STRINGC "You are spotted by a construction worker, and are gently but firmly escorted back to the atrium. \n"
    
;; H Rooms (End)

H1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL H2 ;; Choice 1
    .FILL H5 ;; Choice 2
    .FILL H7 ;; Choice 3
    .STRINGC "\nThe bell rings and you realize you're much too late to get to class now!\n\nLater... \nYou didn't make it to class on time, but you've gone to your professor's office hours to try to get an extension on your late paper. How will you proceed? \n 1) Tell the truth \n 2) Lie (Wisdom) \n 3) Create a distraction so you can slip your paper into the pile without your professor noticing (Charisma) \n"
    
H2: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #5 ;; Stat for dice roll: None
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL H3 ;; Destination if success
    .FILL H4 ;; Destination if failure
    
H3:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #3 ;; 3: End state
    .STRINGC "It worked! Your professor is accepting your paper late with no penalty. \n The End. \n"
    
H4:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #3 ;; 3: End state
    .STRINGC "Unfortunately, your professor has no sympathy for your tale of woe. You decide to drop out of college and live with the geese. \n"
    
H5:
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #4 ;; Stat for dice roll: Wisdom
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #10 ;; Difficulty of check
    .FILL H3 ;; Destination if success
    .FILL H6 ;; Destination if failure
    
H6:
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL H2 ;; Choice 1
    .FILL H11 ;; Choice 2
    .FILL SAD_END ;; Choice 3
    .STRINGC "Your professor didn't believe you. Now what? \n 1) Tell the truth \n 2) Lie again (Wisdom) \n 3) Forget it \n"
    
H7:
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #2 ;; Stat for dice roll: Charisma
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #10 ;; Difficulty of check
    .FILL H8 ;; Destination if success
    .FILL H9 ;; Destination if failure
    
H8:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL WIN ;; Unconditional destination
    .STRINGC "It worked! You make some quick excuses and get out of there before your luck runs out. Who says crime doesn't pay?\n"
    
H9:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #3 ;; 3: End state
    .STRINGC "You got caught -- luckily your professor isn't planning to bring you in front of the honor board, but you'd better be on your best behavior the rest of the semester. Oh, and you're getting a zero on this paper. Oops! \n"
    
H10: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #4 ;; Stat for dice roll: Wisdom
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #12 ;; Difficulty of check
    .FILL H3 ;; Destination if success
    .FILL H11 ;; Destination if failure
    
H11:
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL H2 ;; Choice 1
    .FILL H12 ;; Choice 2
    .FILL SAD_END ;; Choice 3
    .STRINGC "Your professor didn't believe you. Now what? \n 1) Tell the truth \n 2) Lie again (Wisdom) \n 3) Forget it \n"
    
H12: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL SAD_END ;; Unconditional destination
    .STRINGC "This is the third time you've tried this, what did you think would happen? \n"

GOOSE_END:
            .FILL #0 ;; Quality changed: None
            .FILL #0 ;; Amount of change: None
            .FILL #3 ;; 3: End state
            .STRINGC "You live with the geese now. They send your parents a holiday card every year. \n"
            
SAD_END:
        .FILL #0 ;; Quality changed: None
        .FILL #0 ;; Amount of change: None
        .FILL #3 ;; 3: End state
        .STRINGC "You accept the loss of points on your paper and return to Brecon to eat a sad dinner of microwave mac & cheese. \n"

WIN:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #3 ;; 3: End state
    .STRINGC "Congratulations! You have successfully turned in your paper and can continue your student career. You go back to Brecon and take a nap. \n"
    
.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [5]:
%exe

It's the start of a new day at Bryn Mawr College! What class year are you?
In order to choose a year, input the number next to your choice. (Note that if you input an invalid option, it will be interpreted as one of the given options.)
1  Be a freshman.
2  Be a sophomore.
3  Be a junior.
4  Be a senior.

Input a character> 1
1
You are a freshman! +2 Vitality

Here are your stats! When an action you can choose has one of the stats listed next to it, choosing that option will require a check based on that stat -- whether you succeed or fail will affect what happens next! Various things can raise or lower your stats throughout the game. 

Charisma: 0
Wisdom: 0
Vitality: 2

Give your character a name! Type a (2-10 letter) name for your character in the box, then hit return to finish.
nvm
Good morning, nvm!
You are in your room in Brecon! Your paper is written, your teeth are brushed, and your bag is packed. All you need to do is eat lunch, print your paper, and get to class on time to turn

In [114]:
%bp 31dc

Breakpoints
    1)            x31DC: x2E02  LD R7, INTRO_SAVE_R7                      [line: 185]


In [5]:
%dis 3064

Memory disassembled:
           x3064: x0000 - \0
           x3065: x0000 - \0
PUSH:      x3066: x2B9A  LD R5, TOP                                [line: 8]
           x3067: x7140  STR R0, R5, 0                             [line: 9]
           x3068: x1B61  ADD R5, R5, #1                            [line: 10]
           x3069: x7340  STR R1, R5, 0                             [line: 11]
           x306A: x1B61  ADD R5, R5, #1                            [line: 12]
           x306B: x7540  STR R2, R5, 0                             [line: 13]
           x306C: x1B61  ADD R5, R5, #1                            [line: 14]
           x306D: x7740  STR R3, R5, 0                             [line: 15]
           x306E: x1B61  ADD R5, R5, #1                            [line: 16]
           x306F: x7940  STR R4, R5, 0                             [line: 17]
           x3070: x1B61  ADD R5, R5, #1                            [line: 18]
           x3071: x7D40  STR R6, R5, 0                           

In [11]:
%dis 30c8 30DC

Memory disassembled:
           x30C8: xB7D0  STI R3, CHA_IND1                          [line: 131]
           x30C9: x56E0  AND R3, R3, #0                            [line: 132]
           x30CA: x16E1  ADD R3, R3, #1                            [line: 133]
           x30CB: xB7CE  STI R3, WIS_IND1                          [line: 134]
           x30CC: xE1AA  LEA R0, POP                               [line: 135]
           x30CD: xC000  JMP R0                                    [line: 136]
CC_CH4:    x30CE: xE04E  LEA R0, CC_TEXT4                          [line: 137]
           x30CF: x14A4  ADD R2, R2, #4                            [line: 138]
           x30D0: x340A  ST R2, CLASS                              [line: 139]
           x30D1: x3E07  ST R7, CC_SAVE_R7                         [line: 140]
           x30D2: xF024  PUTSP                                     [line: 141]
           x30D3: x2E05  LD R7, CC_SAVE_R7                         [line: 142]
           x30D4: x56E0  AND R3

In [13]:
%dis 31C7 31E0

Memory disassembled:
INTRO:     x31C7: xE018  LEA R0, INTRO_TEXT                        [line: 163]
           x31C8: x3E16  ST R7, INTRO_SAVE_R7                      [line: 164]
           x31C9: xF024  PUTSP                                     [line: 165]
           x31CA: xE0A3  LEA R0, INTRO_CHA                         [line: 166]
           x31CB: xF024  PUTSP                                     [line: 167]
           x31CC: x26B6  LD R3, INTRO_ASCII                        [line: 168]
           x31CD: xA0B6  LDI R0, CHA_IND2                          [line: 169]
           x31CE: x1003  ADD R0, R0, R3                            [line: 170]
           x31CF: xF021  OUT                                       [line: 171]
           x31D0: xE0A3  LEA R0, INTRO_WIS                         [line: 172]
           x31D1: xF024  PUTSP                                     [line: 173]
           x31D2: xA0B2  LDI R0, WIS_IND2                          [line: 174]
           x31D3: x1003  ADD R0

In [15]:
%dis 3244

Memory disassembled:
           x3244: x6168 - 24936 
           x3245: x7070 - 28784 
           x3246: x6E65 - 28261 
           x3247: x2073 - 8307 
           x3248: x656E - 25966 
           x3249: x7478 - 29816 
           x324A: x2021 - 8225 
           x324B: x6156 - 24918 
           x324C: x6972 - 26994 
           x324D: x756F - 30063 
           x324E: x2073 - 8307 
           x324F: x6874 - 26740 
           x3250: x6E69 - 28265 
           x3251: x7367 - 29543 
           x3252: x6320 - 25376 
           x3253: x6E61 - 28257 
           x3254: x7220 - 29216 
           x3255: x6961 - 26977 
           x3256: x6573 - 25971 
           x3257: x6F20 - 28448 
           x3258: x2072 - 8306 
           x3259: x6F6C - 28524 
           x325A: x6577 - 25975 
           x325B: x2072 - 8306 
           x325C: x6F79 - 28537 
           x325D: x7275 - 29301 
           x325E: x7320 - 29472 
           x325F: x6174 - 24948 
           x3260: x7374 - 29556 
           x3261: x7420 - 2

In [16]:
%dis 32A8

Memory disassembled:
           x32A8: x0000 - \0
           x32A9: x0000 - \0
MAX_LEN:   x32AA: x000B  NOOP - (no BR to x32B6) (or 11)           [line: 235]
ROOM_INFO: x32AB: x6947  LDR R4, R5, 7                             [line: 236]
           x32AC: x6576 - 25974 
           x32AD: x7920 - 31008 
           x32AE: x756F - 30063 
           x32AF: x2072 - 8306 
           x32B0: x6863 - 26723 
           x32B1: x7261 - 29281 
           x32B2: x6361 - 25441 
           x32B3: x6574 - 25972 
           x32B4: x2072 - 8306 
           x32B5: x2061 - 8289 
           x32B6: x616E - 24942 
           x32B7: x656D - 25965 
           x32B8: x2021 - 8225 
           x32B9: x7954 - 31060 
           x32BA: x6570 - 25968 
           x32BB: x6120 - 24864 
           x32BC: x2820 - 10272 
           x32BD: x2D32 - 11570 
           x32BE: x3031 - 12337 
           x32BF: x6C20 - 27680 
           x32C0: x7465 - 29797 
           x32C1: x6574 - 25972 
           x32C2: x2972 - 10610 
        

In [5]:
ROOM_UINPUT: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL ROOM_VARIABLE
    .FILL ROOM_RANDOM
    .FILL SWORD_ROOM
    .STRINGC "Here we are in an example room. Enter 1 to TEST UR SWORD, 2 to go to the random room, or 3 to GET A SWORD\n"
    
ROOM_RANDOM: 
    .FILL #2 ;; Quality changed: HP
    .FILL #2;; Amount of change: 2
    .FILL #1 ;; 1: Based on dice roll
    .FILL #3 ;; Stat for dice roll: Vitality
    .FILL #8 ;; Difficulty of check
    .FILL ROOM_UINPUT ;; Destination if success
    .FILL ROOM_RANDOM ;; Destination if failure
    
MESSAGE_RM: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL ROOM_UINPUT
    .STRINGC "You passed the sword check!\n" 
    
SWORD_ROOM:
    .FILL #5 ;; Quality changed: Sword
    .FILL #1 ;; Amount of change: One
    .FILL #0 ;; 0: Based on user input
    .FILL ROOM_UINPUT
    .FILL ROOM_UINPUT
    .FILL ROOM_UINPUT ;; Three choices all lead to user input room
    .STRINGC "SWORD ROOM SWORD ROOM SWORM never come back or u will lose your sword\n"
    
ROOM_VARIABLE:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #2 ;; 2: Based on variable
    .FILL SWORD ;; VAR_ADDRESS ;; What variable to check
    .FILL #1 ;; VALUE       ;; Minimum value that counts as a success
    .FILL MESSAGE_RM ;; Destination if success
    .FILL SWORD_DEATH ;; Destination if failure
    
SWORD_DEATH: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #3 ;; 3: End state
    .STRINGC "Your SWORDLESSNESS was no match for the TEST OF SWORDS\n"
    
ENDGAME: ;; Note: doesn't HAVE to be an ending state, could be uinput and lead to others
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #3 ;; 3: End state
    .STRINGC "You ran out of time!\n"


Assemble error
    memory x5016

Bad label: "SWORD"

